# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [1]:
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload

from concept_dependency_graph import ConceptDependencyGraph
import data_generator as dg
import simple_mdp as sm

In [2]:
n_concepts = 5
concept_tree = sm.create_custom_dependency()
n_students = 100000
seqlen = 3
policy = 'random'
filename = 'test-n{}-l{}-{}.pickle'.format(n_students, seqlen, policy)

In [3]:
print ("Initializing synthetic data sets...")
dg.generate_data(concept_tree, n_students=n_students, seqlen=seqlen, policy=policy, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 100000 students with behavior policy random and sequence length 3.
Data generation completed. 


In [3]:
import dynamics_model_class as dmc
import numpy as np
import dataset_utils

In [18]:
# test_model hidden=16
# test_model_small hidden=5
# test_model_mid hidden=10
dmodel = dmc.DynamicsModel(model_id="test_model_small", timesteps=seqlen, load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test_model_small/
Checkpoint directory path: ../checkpoints/test_model_small/
Model loaded.


In [5]:
# load toy data
data = dataset_utils.load_data(filename='{}{}'.format(dg.SYN_DATA_DIR, filename))
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

In [6]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])

In [19]:
dmodel.train(train_data, n_epoch=7, load_checkpoint=False)

Training Step: 9848  | total loss: 0.01479 | time: 17.191s
| Adam | epoch: 007 | loss: 0.01479 -- iter: 89984/90000
Training Step: 9849  | total loss: 0.01478 | time: 18.302s
| Adam | epoch: 007 | loss: 0.01478 | val_loss: 0.01498 -- iter: 90000/90000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [20]:
preds = dmodel.predict(input_data_[:1,:2, :])
print len(preds)
print len(preds[0])
print preds[0]

1
3
[[0.7307418584823608, 0.48607394099235535, 0.49976059794425964, 0.014471620321273804, 0.016842937096953392], [0.8806973099708557, 0.4929755628108978, 0.48517948389053345, 0.005949639715254307, 0.008941549807786942], [0.0, 0.0, 0.0, 0.0, 0.0]]


In [21]:
rnnsim = dmc.RnnStudentSim(dmodel)

In [22]:
from mcts import *
actix = 0
actvec = np.zeros((n_concepts,))
actvec[actix] = 1
act = StudentAction(actix, actvec)
print rnnsim.sequence
print rnnsim.sample_observations()

[]
None


In [23]:
rnnsim.advance_simulator(act, 1)


In [24]:
print rnnsim.sequence
print rnnsim.sample_observations()
print rnnsim.sample_reward()

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]
[0.7307418584823608, 0.48607394099235535, 0.49976059794425964, 0.014471620321273804, 0.016842937096953392]
1.74789095484
